In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import warnings
import statsmodels.api as sm
import os
warnings.filterwarnings("ignore")
os.chdir("../../db")
print(os.getcwd())

/Users/test/risksoft/TS/db


In [6]:
pct_close = pd.read_parquet("tw/pdata/close_pct.parquet")
pct_close_w = pct_close.resample("W-FRI").sum()

indexes = pct_close_w.index
tickers = pct_close_w.iloc[:, :].columns
indices = indexes[52:104]
forecast = np.zeros((len(indices), len(tickers)))
print(indices[0], indices[-1])
for index, date in enumerate(tqdm(indices, desc=f"arma model for top {len(tickers)}, enumerate by da")):
    pct_train = pct_close_w.loc[indexes[index]:date]
    for i in range(len(tickers)):
        series = pct_train[tickers[i]]
        if series.isna().any():
            prediction = 0
            print(tickers[i], "failed at", date)
            continue
            ###### ARIMA ma
        else:
            model = sm.tsa.arima.ARIMA(series, order=(2, 0, 0))
            model_fit = model.fit()
            prediction = model_fit.forecast().values[0]
        forecast[index, i] = prediction
factor = pd.DataFrame(forecast, columns=tickers, index=indices)
